In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

In [16]:
#取得最高價格
def GetMaxPrice(PriceList):
    PriceSplit=[]
    for Price in PriceList:    
        for onePrice in (str(Price.text).replace('￥','').replace(',','').replace('-','').split('～')):
            if len(onePrice)>0:
                PriceSplit.append(int(onePrice))
            else:
                PriceSplit.append(0)
    PriceSplit.sort(reverse=True)
    return PriceSplit[0]

In [22]:
#Parser Web資料
def ParserWeb(breakMax):
    Name_list=[]
    Url_list=[]
    Rating_list=[]
    PriceList_list=[]
    ImageLink_list=[]
    MaxPrice_list=[]
    page=1
    
    while True:
        try:
            url='https://tabelog.com/tw/rstLst/'+str(page)+'/?LstCatD=RC0101&LstCat=RC01&Cat=RC'
            header={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.62'}
            response=requests.get(url,headers=header)
            print(url+":"+str(response))
            info=response.content.decode(encoding='utf-8')
            soup = BeautifulSoup(info, 'html.parser')

            print(f'Now Page:{page}')
            page=page+1
            
            #餐廳清單
            restaurants=soup.find_all('li',class_="list-rst js-list-item")

            #切割欄位
            for restaurant in restaurants:
                Name=restaurant.find('small',class_='list-rst__name-ja').text
                Url=restaurant.find('a',class_='list-rst__reviews-target').get('href')
                Rating=restaurant.find('b',class_='c-rating__val').text
                PriceList=restaurant.find_all('span',class_='c-rating__val')
                ImageLink=restaurant.find('img',class_='c-img c-img--frame').get('src')
                MaxPrice=GetMaxPrice(PriceList)

                Name_list.append(Name)
                Url_list.append(Url)
                Rating_list.append(Rating)
                PriceList_list.append(PriceList)
                ImageLink_list.append(ImageLink)
                MaxPrice_list.append(MaxPrice)

            if page>breakMax:
                break  
            
            randomSecond=random.randint(10,15)
            print(f'Sleep:{randomSecond}')
            time.sleep(randomSecond)                  
        except:
            print(f'Error發生')
    return Name_list,Url_list,Rating_list,PriceList_list,ImageLink_list,MaxPrice_list

In [23]:
#呼叫抓取
ParserWebRtn=ParserWeb(1)

https://tabelog.com/tw/rstLst/1/?LstCatD=RC0101&LstCat=RC01&Cat=RC:<Response [200]>
Now Page:1
Sleep:15
https://tabelog.com/tw/rstLst/2/?LstCatD=RC0101&LstCat=RC01&Cat=RC:<Response [200]>
Now Page:2
Sleep:10
https://tabelog.com/tw/rstLst/3/?LstCatD=RC0101&LstCat=RC01&Cat=RC:<Response [200]>
Now Page:3


In [24]:
#輸出到DataFrame
df=pd.DataFrame(columns=['Name','Url','Rating','PriceList','ImageLink','MaxPrice'])
df['Name']=ParserWebRtn[0]
df['Url']=ParserWebRtn[1]
df['Rating']=ParserWebRtn[2]
df['PriceList']=ParserWebRtn[3]
df['ImageLink']=ParserWebRtn[4]
df['MaxPrice']=ParserWebRtn[5]